# numpy 中的 as_strided 函数

2019 年 5 月 17 日: [卷积算法另一种高效实现，as_strided详解](https://zhuanlan.zhihu.com/p/64933417)

## 关于 strides

`np.lib.stride_tricks.as_strided` 函数比较复杂, 主要是其中的 `strides` 参数需要设置.

实际上, 对于每一个 numpy 数组, 它都有一个名为 `strides` 的属性, 

> strides是numpy数组对象的一个属性，官方手册给出的解释是跨越数组各个维度所需要经过的字节数（bytes）
>
> 例如，跨越第1维：从a[x,0]到a[x,1]需要经过4个字节，为什么是4个？因为a的数据类型是int32，正好占4个字节
> 
> 跨越第0维：从a[0,x]到a[1,x]的话，需要经过12（3x4）个字节，因为a[0]，也就是a的一行有3个数据，占据12个字节

比如对于

```python
a = array([[0, 1, 2],
           [3, 4, 5],
           [6, 7, 8]], dtype=np.int32)
```

跨越 `axis=1` 这个维度, 比如从 `0` 跨到 `1`, 需要经过 4 个字节; 而跨越 `axis=0` 这个维度, 比如从 `0` 跨越到 `3`, 需要经过 `4 * 3 = 12` 个字节, 因为 `a` 每行有 3 个数据. 输出数组 `a` 的 `strides` 属性, 得到:

```python
print(a.strides)

(12, 4)
```

结果是每个维度上的 strides.

在上面参考资料中, 我们主要用 `np.lib.stride_tricks.as_strided` 来处理卷积 (为了高效实现 `im2col`), 对于一个 `shape=(m, n, k, d)` 的矩阵 $X$, 可以得到它的 strides 为 (假设字节大小为 `byte`):

```python
(n * k * d * byte, k * d * byte, d * byte, byte)
```

之所以将 $X$ 的 shape 写成上面的形式, 是为了方便推广, 比如说对于 `shape=(N, C, H, W)` 大小的矩阵 $x$, 它的 strides 为:

```python
(C * H * W * byte, H * W * byte, W * byte, byte)
```

然而, 需要注意一点的是, 如果使用了 `transpose` 方法, 那么 `strides` 就会有所变化, 比如对 $x$ 使用 `x.transpose(0, 2, 3, 1)`, 那么结果的 `shape` 就变成了 `(N, H, W, C)`, 此时 `x.transpose(0, 2, 3, 1)` 的 `strides` 就变成了:

```python
(C * H * W * byte, W * byte, byte, H * W * byte)
```

## 关于 as_strided 函数

下面看 `as_strided` 的作用. 比如对于 $X$ 的 `shape=(c, h, w)=(2, 3, 3)`, 假设 kernel 的 `shape=(kh, kw)=(2, 2)`, 那么可以使用 `as_strided` 将 $X$ 转换为矩阵 $A$:

$$
X = \left[
    \left[\begin{matrix}
    0 & 1 & 2 \\
    3 & 4 & 5 \\
    6 & 7 & 8\end{matrix}\right]\quad
    \left[\begin{matrix}
    9 & 10 & 11 \\
    12 & 13 & 14 \\
    15 & 16 & 17\end{matrix}\right]
    \right]
$$

通过使用 `as_strided`, 得到 $A$:

$$
A = \left[
    \left[\begin{matrix}
        \left[\begin{matrix}0 & 1 \\ 3 & 4\end{matrix}\right]&
        \left[\begin{matrix}1 & 2 \\ 4 & 5\end{matrix}\right] \\
        \left[\begin{matrix}3 & 4 \\ 6 & 7\end{matrix}\right]&
        \left[\begin{matrix}4 & 5 \\ 7 & 8\end{matrix}\right]
    \end{matrix}\right]\quad
    \left[\begin{matrix}
        \left[\begin{matrix}9 & 10 \\ 12 & 13\end{matrix}\right] &
        \left[\begin{matrix}10 & 11 \\ 13 & 14\end{matrix}\right] \\
        \left[\begin{matrix}12 & 13 \\ 15 & 16\end{matrix}\right]&
        \left[\begin{matrix}13 & 14 \\ 16 & 17\end{matrix}\right]
    \end{matrix}\right]
\right]
$$

$A$ 的 $shape$ 是多少呢? 假设 kernel 在 $X$ 上移动, stride 的大小为 $s$, 那么 $A$ 的 `shape=(c, oh, ow, h, w)`, 其中:

$$
\begin{aligned}
oh &= \frac{(h - kh)}{ s} + 1 \\
ow & = \frac{(w - kw)}{ s} + 1
\end{aligned}
$$

下面再来看 $A$ 的 strides, $A$ 的 `dims=(0, 1, 2, 3, 4)`, 那么:

+ `dim=4` (即 `w` 对应的 axis) 对应的 strides 为: `byte`
+ `dim=3` (即 `h` 对应的 axis) 对应的 strides 为: `w * byte`
+ `dim=2` (即 `ow` 对应的 axis) 对应的 strides 为 `byte`; 这里要理解为什么是 `byte`, 因为 `ow` 这个维度表示的是: kernel 在 $X$ 上水平移动, 比如从:

$$
\left[\begin{matrix}0 & 1 \\ 3 & 4\end{matrix}\right]\quad\text{移动到}\quad\left[\begin{matrix}1 & 2 \\ 4 & 5\end{matrix}\right]
$$

它们之间对应位置上的元素相差一个 `byte`.
+ `dim=1` (即 `oh` 对应的 axis) 对应的 strides 为 `w * byte`; 和上面同理, 比如 kernel 在 $X$ 上竖直移动, 比如从:
$$
\left[\begin{matrix}0 & 1 \\ 3 & 4\end{matrix}\right]\quad\text{移动到}\quad\left[\begin{matrix}3 & 4 \\ 6 & 7\end{matrix}\right]
$$

它们之间对应位置上的元素相差一个 `w * byte`.
+ `dim=0` (即 `c` 对应的 axis) 对应的 strides 为 `h * w * byte`, 相当于从一个通道移动到下一个通道.

## 更多的情况

+ $X$ 的 `shape=(N, C, H, W)` 的情况

$X$ 的 strides 为 `(C * H * W * byte, H * W * byte, W * byte, byte)`, 此时 $A$ 的 `shape=(N, C, oh, ow, kh, kw)`, 对应的 strides 为 `(C * H * W * byte, H * W * byte, W * byte, byte, W * byte, byte)`

+ $X$ 的 `shape=(N, H, W, C)` 的情况

$X$ 的 strides 为 `(H * W * C * byte, W * C * byte, W * byte, byte)`, 此时 $A$ 的 `shape=(N, oh, ow, kh, kw, C)`, 对应的 strides 为 `(H * W * C * byte, W * C * byte, C * byte, W * C * byte, C * byte, byte)`

总结规律:

+ `oh` 这个轴的 strides 和 `h` 这个轴的 strides 相等
+ `ow` 这个轴的 strides 和 `w` 这个轴的 strides 相等
+ `N` 和 `C` 这两个轴的 strides 不变.

In [15]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

def split_by_strides_typeI(X, kh, kw, s):
    """
    X 为 (N, C, H, W) 的情况, 
    
    如果 X 的 strides 为 (N1, C1, H1, W1), 
    
    那么 A 的 (考虑 stride 这个参数, 即函数中的 `s`)
    shape 为: (N, C, oh, ow, kh, kw)
    strides 为: (N1, C1, H1 * s, W1 * s, H1, W1)
    """
    N, C, H, W = X.shape
    oh = (H - kh) // s + 1
    ow = (W - kw) // s + 1
    strides = (*X.strides[:-2], X.strides[-2]*s, X.strides[-1]*s, *X.strides[-2:])
    A = as_strided(X, shape=(N,C,oh,ow,kh,kw), strides=strides)
    return A
    

def split_by_strides_typeII(X, kh, kw, s):
    """
    X 为 (N, H, W, C) 的情况, 
    
    如果 X 的 strides 为 (N1, H1, W1, C1), 
    
    那么 A 的 (考虑 stride 这个参数, 即函数中的 `s`)
    shape 为: (N, oh, ow, kh, kw, C)
    strides 为: (N1, H1 * s, W1 * s, H1, W1, C1)
    """
    N, H, W, C = X.shape
    oh = (H - kh) // s + 1
    ow = (W - kw) // s + 1
    shape = (N, oh, ow, kh, kw, C)
    strides = (X.strides[0], X.strides[1]*s, X.strides[2]*s, *X.strides[1:])
    A = as_strided(X, shape=shape, strides=strides)
    return A

## 此外

不得不说的一点, 如果只记忆上面的结论, 对 `as_strided` 的理解就不深刻. 要记的东西是, `as_strided` 以一种新的视角来创建一个矩阵. 即底层的内存中的东西不变, 在将矩阵展示给你看的时候, 将底层的元素进行某种排列. 排列的方法就是通过 `shape` 和 `strides` 来设置, 当确定 `shape` 之后, 再指定 `strides` 去对应的查找元素. 如下面的例子, 如果 A 发生了变化, X 也同样会发生变化.

In [16]:
X = np.arange(4).reshape(1, 1, 2, 2)
A = split_by_strides_typeI(X, 1, 1, 1)
print(A.shape)
A[0, 0, 0, 0, 0, 0] = 100
X

(1, 1, 2, 2, 1, 1)


array([[[[100,   1],
         [  2,   3]]]])

## bonus: 对于 transpose 的认识

看完参考链接中的博客, 其中有一段对于 `as_strided` 的认识:

> 因为A不是一个新的数组，而是X的一种视图（View），所以，strides的理解是，在矩阵X的基础上，按照给定的strides，来切割出一个符合给定shape的视图

可以理解为, $A$ 和 $X$ 是对同一段内存中的数据表现出的两种不同的形式. 具体可以通过对 `transpose` 这个函数来进行理解.

In [6]:
X = np.arange(6).reshape(2, 3)
Y = X.transpose(1, 0) # shape: (3, 2)
Y[0, 0] = 10
print(X[0, 0]) # 当 Y 发生变化之后, X 也会发生变化

10


In [7]:
X.strides, Y.strides # byte=8, 因为 dtype 默认为 np.int64

((24, 8), (8, 24))

注意上面 X 和 Y 的 stides, Y 的 strides 也被转置了. 如果 Y 是按照 `np.arange` 生成的, 那么其 strides 就不是这个样子

In [8]:
Z = np.arange(6).reshape(3, 2)
Z.strides # 虽然 shape 和 Y 一样, 但是 strides 不同.

(16, 8)